In [1]:
import pandas as pd 
import datetime as dt
import matplotlib.pyplot as plt
from lifetimes import BetaGeoFitter
from lifetimes import GammaGammaFitter
from lifetimes.plotting import plot_period_transactions
from sklearn.preprocessing import MinMaxScaler

In [2]:
Coffee = pd.read_csv("/Users/ASUS/Desktop/BI_Final/Data/sales2022.csv")
Coffee

,transaction_id,transaction_date,transaction_time,store_id,staff_id,customer_id,product_id,quantity_sold,unit_price,promo_item_yn,revenue
0,390,1/1/2022,7:01:00,5,12,5658,22,2,2.0,N,4.0
1,1228,1/1/2022,7:02:00,5,30,5197,22,1,2.0,N,2.0
2,1644,1/1/2022,7:03:00,5,12,5247,22,2,2.0,N,4.0
3,1338,1/1/2022,7:06:00,5,30,5465,32,2,3.0,N,6.0
4,2257,1/1/2022,7:06:00,5,30,5627,22,1,2.0,N,2.0
...,...,...,...,...,...,...,...,...,...,...,...
140440,626,4/29/2022,19:57:00,8,15,8440,22,1,2.0,N,2.0
140441,626,4/29/2022,19:57:00,8,15,8440,77,1,3.0,N,3.0
140442,3849,4/29/2022,19:57:00,3,14,1000,52,2,2.5,N,5.0
140443,1578,4/29/2022,19:57:00,3,14,1000,51,2,3.0,N,6.0


In [3]:
def outliers_threshold(dataframe, variable):
    quartile1 = dataframe[variable].quantile(0.25)
    quartile3 = dataframe[variable].quantile(0.75)
    interquantile_range = quartile3 - quartile1
    up_limit = quartile3 + 1.5 * interquantile_range
    low_limit = quartile1 - 1.5 * interquantile_range
    return low_limit, up_limit

In [4]:
def replace_with_threshold(dataframe, variable):
    low_limit, up_limit = outliers_threshold(dataframe, variable)
    dataframe.loc[(dataframe[variable] < low_limit), variable] = low_limit
    dataframe.loc[(dataframe[variable] > up_limit), variable] = up_limit

In [5]:
replace_with_threshold(Coffee, "revenue")
replace_with_threshold(Coffee, "unit_price")

In [6]:
cltv_c = Coffee.groupby("customer_id").agg({"transaction_id": lambda x: x.nunique(),
                                        "revenue": lambda x: x.sum()})
cltv_c

,transaction_id,revenue
customer_id,,
1,6,85.40
2,16,263.60
3,13,498.00
4,7,83.30
5,6,82.50
...,...,...
8498,7,126.00
8499,10,156.20
8500,17,273.25


In [7]:
cltv_c.columns = ["total_transaction", "total_price"]

### Average Order Value = Total Price / Total Transaction

In [8]:
cltv_c["average_order_value"] = cltv_c["total_price"] / cltv_c["total_transaction"]
cltv_c["average_order_value"]


customer_id
1       14.233333
2       16.475000
3       38.307692
4       11.900000
5       13.750000
          ...    
8498    18.000000
8499    15.620000
8500    16.073529
8501    10.875000
8600    50.515944
Name: average_order_value, Length: 2250, dtype: float64

### Purchase Frequency = Total Transaction / Total Number of Customers

In [9]:
cltv_c["purchase_frequency"] = cltv_c["total_transaction"] / cltv_c.shape[0]
cltv_c["purchase_frequency"] 


customer_id
1       0.002667
2       0.007111
3       0.005778
4       0.003111
5       0.002667
          ...   
8498    0.003111
8499    0.004444
8500    0.007556
8501    0.002667
8600    1.004889
Name: purchase_frequency, Length: 2250, dtype: float64

### Customer Value = Average Order Value * Purchase Frequency

In [10]:
cltv_c["customer_value"] = cltv_c["average_order_value"] * cltv_c["purchase_frequency"]
cltv_c["customer_value"]


customer_id
1        0.037956
2        0.117156
3        0.221333
4        0.037022
5        0.036667
          ...    
8498     0.056000
8499     0.069422
8500     0.121444
8501     0.029000
8600    50.762911
Name: customer_value, Length: 2250, dtype: float64

### Repeat Rate = Number of customers who made multiple purchases / Total Number of Customers

In [11]:
repeat_rate = cltv_c[cltv_c["total_transaction"] > 1].shape[0] / cltv_c.shape[0]
repeat_rate


0.9951111111111111

In [12]:
churn_rate = 1 - repeat_rate
churn_rate

0.004888888888888943

In [13]:
cltv_c["profit_margin"] = cltv_c["total_price"] * 0.10
cltv_c["profit_margin"]


customer_id
1           8.540
2          26.360
3          49.800
4           8.330
5           8.250
          ...    
8498       12.600
8499       15.620
8500       27.325
8501        6.525
8600    11421.655
Name: profit_margin, Length: 2250, dtype: float64

In [14]:
cltv_c["cltv"] = cltv_c["customer_value"] / churn_rate * cltv_c["profit_margin"]
cltv_c["cltv"]


customer_id
1       6.630145e+01
2       6.316815e+02
3       2.254582e+03
4       6.308082e+01
5       6.187500e+01
            ...     
8498    1.443273e+02
8499    2.218040e+02
8500    6.787778e+02
8501    3.870511e+01
8600    1.185947e+08
Name: cltv, Length: 2250, dtype: float64

In [15]:
cltv_c.sort_values(by="cltv", ascending=False)
cltv_c["segment"] = pd.qcut(cltv_c["cltv"], 4, labels=["D", "C", "B", "A"])

In [16]:
cltv_c.groupby("segment").agg({"sum", "mean", "count"})


total_transaction                  total_price                    \
                      sum count       mean         sum count        mean   
segment                                                                    
D                    2736   563   4.859680    36758.78   563   65.290906   
C                    4180   562   7.437722    63200.45   562  112.456317   
B                    5311   563   9.433393    86156.39   563  153.030888   
A                   15064   562  26.804270   445344.40   562  792.427758   

        average_order_value                  purchase_frequency  ...  \
                        sum count       mean                sum  ...   
segment                                                          ...   
D               7945.533353   563  14.112848           1.216000  ...   
C               8850.259619   562  15.747793           1.857778  ...   
B               9514.420528   563  16.899504           2.360444  ...   
A              10040.802104   562  17.866196           6.695111  ...   

                  customer_value                 profit_margin        \
             mean            sum count      mean           sum count   
segment                                                                
D        0.002160      16.337236   563  0.029018      3675.878   563   
C        0.003306      28.089089   562  0.049981      6320.045   562   
B        0.004193      38.291729   563  0.068014      8615.639   563   
A        0.011913     197.930844   562  0.352190     44534.440   562   

                            cltv                       
              mean           sum count           mean  
segment                                                
D         6.529091  2.354849e+04   563      41.826799  
C        11.245632  6.530720e+04   562     116.204984  
B        15.303089  1.207747e+05   563     214.519837  
A        79.242776  3.081270e+08   562  548268.726876  

[4 rows x 21 columns]

In [17]:
cltv_c.to_csv("cltv_c.csv")


In [18]:
def calculate_cltv(dataframe):
    cltv_c = dataframe.groupby("customer_id").agg({"transaction_id": lambda x: x.nunique(),
                                                    "revenue": lambda x: x.sum()})
    cltv_c.columns = ["total_transaction", "total_price"]
    cltv_c["average_order_value"] = cltv_c["total_price"] / cltv_c["total_transaction"]
    cltv_c["purchase_frequency"] = cltv_c["total_transaction"] / cltv_c.shape[0]
    cltv_c["customer_value"] = cltv_c["average_order_value"] * cltv_c["purchase_frequency"]
    repeat_rate = cltv_c[cltv_c["total_transaction"] > 1].shape[0] / cltv_c.shape[0]
    churn_rate = 1 - repeat_rate
    cltv_c["profit_margin"] = cltv_c["total_price"] * 0.10
    cltv_c["cltv"] = cltv_c["customer_value"] / churn_rate * cltv_c["profit_margin"]
    cltv_c.sort_values(by="cltv", ascending=False)
    cltv_c["segment"] = pd.qcut(cltv_c["cltv"], 4, labels=["D", "C", "B", "A"])
    return cltv_c

In [19]:
df1= calculate_cltv(Coffee)

In [20]:
df1.to_csv("CLV.csv")

In [21]:
Coffee["transaction_date"] = pd.to_datetime(Coffee["transaction_date"])
today_date = dt.datetime(2022, 5, 1)

In [22]:
cltv_df = Coffee.groupby("customer_id").agg({"transaction_date": [lambda date: (date.max() - date.min()).days,
                                                              lambda date: (today_date - date.min()).days],
                                         "transaction_id": lambda num: num.nunique(),
                                         "revenue": lambda x: x.sum()})

In [23]:
cltv_df 

transaction_date            transaction_id    revenue
                  <lambda_0> <lambda_1>       <lambda>   <lambda>
customer_id                                                      
1                        117        119              6      85.40
2                        115        120             16     263.60
3                        112        119             13     498.00
4                        114        119              7      83.30
5                        118        120              6      82.50
...                      ...        ...            ...        ...
8498                     114        118              7     126.00
8499                     117        119             10     156.20
8500                     113        118             17     273.25
8501                     116        119              6      65.25
8600                     112        114           2261  114216.55

[2250 rows x 4 columns]

In [24]:
cltv_df.columns = cltv_df.columns.droplevel(0)
cltv_df.columns = ["recency", "T", "frequency", "monetary"]
cltv_df 

,recency,T,frequency,monetary
customer_id,,,,
1,117,119,6,85.40
2,115,120,16,263.60
3,112,119,13,498.00
4,114,119,7,83.30
5,118,120,6,82.50
...,...,...,...,...
8498,114,118,7,126.00
8499,117,119,10,156.20
8500,113,118,17,273.25


In [25]:
cltv_df["monetary"] = cltv_df["monetary"] / cltv_df["frequency"]


In [26]:
cltv_df = cltv_df[(cltv_df["frequency"] > 1)]


In [27]:
cltv_df["T"] = cltv_df["T"] / 7
cltv_df["recency"] = cltv_df["recency"] / 7

In [28]:
bgf = BetaGeoFitter(penalizer_coef=0.001)
bgf.fit(cltv_df["frequency"], cltv_df["recency"], cltv_df["T"])

<lifetimes.BetaGeoFitter: fitted with 2239 subjects, a: 0.00, alpha: 1.80, b: 0.64, r: 1.33>

In [29]:
bgf.conditional_expected_number_of_purchases_up_to_time(4,
                                                        cltv_df["frequency"],
                                                        cltv_df["recency"],
                                                        cltv_df["T"]).sort_values(ascending=False).head(10)

customer_id
8285    5.558100
8101    5.470731
8384    5.135872
8297    5.135872
548     4.924732
8118    4.924732
8341    4.924700
8364    4.785556
8019    4.749362
8072    4.713644
dtype: float64

In [30]:
cltv_df["expected_purchase_4_week"] = bgf.conditional_expected_number_of_purchases_up_to_time(4,
                                                        cltv_df["frequency"],
                                                        cltv_df["recency"],
                                                        cltv_df["T"])

In [31]:
bgf.conditional_expected_number_of_purchases_up_to_time(12,
                                                        cltv_df["frequency"],
                                                        cltv_df["recency"],
                                                        cltv_df["T"]).sort_values(ascending=False).head(10)

customer_id
8285    16.673105
8101    16.410991
8384    15.406507
8297    15.406507
548     14.773129
8118    14.773129
8341    14.773035
8364    14.355617
8019    14.247050
8072    14.139908
dtype: float64

In [32]:
cltv_df["expected_purchase_3_month"] = bgf.conditional_expected_number_of_purchases_up_to_time(12,
                                                        cltv_df["frequency"],
                                                        cltv_df["recency"],
                                                        cltv_df["T"])

In [33]:
bgf.predict(12, cltv_df["frequency"], cltv_df["recency"], cltv_df["T"]).sum()

14567.962147380107

In [34]:
ggf = GammaGammaFitter(penalizer_coef=0.01)
ggf.fit(cltv_df["frequency"], cltv_df["monetary"])

<lifetimes.GammaGammaFitter: fitted with 2239 subjects, p: 4.45, q: 1.65, v: 4.33>

In [35]:
cltv = ggf.customer_lifetime_value(bgf,
                                   cltv_df["frequency"],
                                   cltv_df["recency"],
                                   cltv_df["T"],
                                   cltv_df["monetary"],
                                   time=3, 
                                   freq="W",
                                   discount_rate=0.01)

In [36]:

cltv_final = cltv_df.merge(cltv, on="customer_id", how="left")
cltv_final.sort_values(by="clv", ascending=False).head(10)

,recency,T,frequency,monetary,expected_purchase_4_week,expected_purchase_3_month,clv
customer_id,,,,,,,
3,16.000000,17.000000,13,38.307692,3.047612,9.142149,371.978764
8341,16.571429,17.142857,22,19.909091,4.924700,14.773035,314.199933
8285,16.857143,17.142857,25,15.992000,5.558100,16.673105,285.335650
8410,16.857143,17.142857,18,21.444444,4.080300,12.240007,280.359260
8364,15.714286,16.857143,21,18.230952,4.785556,14.355617,279.892610
8144,16.142857,17.142857,16,23.700000,3.657951,10.973044,277.550136
8322,15.857143,16.142857,18,19.580556,4.307652,12.921970,270.539495
8459,16.285714,17.142857,14,26.057143,3.235767,9.706578,269.704925
5026,16.000000,16.857143,15,23.610000,3.499646,10.498152,264.580909


In [45]:
cltv_final.to_csv('CLVL_Prediction.csv')